<a href="https://colab.research.google.com/github/wFairmanOlin/HAUCS/blob/main/GPS_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import plotly.express as px

In [49]:
### ONLY RUN ONCE! ###
# #download data from firebase
# cred = credentials.Certificate("/content/haucs-monitoring-firebase-adminsdk-wt1pb-1291f2083e.json")
# firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})
try:
    print(cred)
except:
    cred = credentials.Certificate("../Biomass/pc_basestation/fb_key.json")
    firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})

In [50]:
## ONLY RUN TO REFRESH DATA! ###
ref = db.reference('/LH_Farm')
data = ref.get()

### RUN TO SAVE JSON FILE LOCALLY ###
with open('latest_data.json', 'w') as f:
    json.dump(data, f)

In [44]:
gps_df = pd.DataFrame(data['bathymetry'], index=['RSSI', 'lat', 'lng']).T.reset_index() #was data['gps']
gps_df.columns = ['timestamp', 'RSSI', 'lat', 'lng']

gps_df.head()

# Split the concatenated values in the lat and lng columns
gps_df[['lat', 'lng']] = gps_df[['lat', 'lng']].apply(lambda x: x.str.split().str[0])

# Convert the necessary columns to numeric type
gps_df['RSSI'] = pd.to_numeric(gps_df['RSSI'])
gps_df['lat'] = pd.to_numeric(gps_df['lat'])
gps_df['lng'] = pd.to_numeric(gps_df['lng'])

# Convert the timestamp column to datetime type
gps_df['timestamp'] = pd.to_datetime(gps_df['timestamp'], format='%Y%m%d_%H:%M:%S')
start = datetime(year=2024, month=5, day=22)
end = datetime(year=2024, month=5, day=23)
gps_df = gps_df[(gps_df['timestamp'] > start) & (gps_df['timestamp'] < end)]

gps_df.head()

,timestamp,RSSI,lat,lng
1473,2024-05-22 14:33:13,-141,37.705913,-89.468673
1474,2024-05-22 14:33:31,-140,37.705913,-89.468673
1475,2024-05-22 14:33:43,-136,37.706668,-89.468666
1476,2024-05-22 14:33:46,-136,37.706668,-89.468673
1477,2024-05-22 14:33:51,-138,37.706676,-89.468673


In [56]:
gps_df = pd.DataFrame(data['bathymetry'], index=['lat', 'lng']).T.reset_index() #was data['gps']
gps_df.columns = ['timestamp', 'lat', 'lng']

gps_df.head()

# Split the concatenated values in the lat and lng columns
gps_df[['lat', 'lng']] = gps_df[['lat', 'lng']].apply(lambda x: x.str.split().str[0])

# Convert the necessary columns to numeric type
gps_df['lat'] = pd.to_numeric(gps_df['lat'])
gps_df['lng'] = pd.to_numeric(gps_df['lng'])

# Convert the timestamp column to datetime type
gps_df['timestamp'] = pd.to_datetime(gps_df['timestamp'], format='%Y%m%d_%H:%M:%S')
start = datetime(year=2024, month=5, day=24)
end = datetime(year=2024, month=5, day=25)
gps_df = gps_df[(gps_df['timestamp'] > start) & (gps_df['timestamp'] < end)]

gps_df.head()

,timestamp,lat,lng
102,2024-05-24 14:30:15,37.701896,-89.468101
103,2024-05-24 14:30:26,37.701896,-89.468101
104,2024-05-24 14:30:37,37.701889,-89.468093
105,2024-05-24 14:30:48,37.701889,-89.468093
106,2024-05-24 14:30:59,37.701885,-89.468093


In [57]:
# px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(gps_df,
                        lat="lat",
                        lon="lng",
                        hover_name="timestamp",
                        mapbox_style='carto-positron',
                        zoom=1)

# Set mapbox token and layout
fig.update_layout(
    width=800,
    height=500,
    mapbox=dict(
        accesstoken='YOUR_MAPBOX_TOKEN',
        style='open-street-map',
        center=dict(lat=gps_df['lat'].mean(), lon=gps_df['lng'].mean()),
        zoom=14,
        
    )
)


/Users/wfairman/opt/anaconda3/envs/lab2/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

